In [69]:
import os
import re
import csv
import random
import sqlite3
from typing import List, Optional
from pathlib import Path
from urllib.request import urlopen, urlretrieve
from urllib.parse import urlparse, urljoin

from bs4 import BeautifulSoup

In [14]:
random.seed(42)

In [3]:
def get_absolute_url(base_url: str, source: str) -> Optional[str]:
    is_absolute = source.startswith(("http://", "https://"))
    has_external_domain = is_absolute and base_url not in source

    if has_external_domain:
        return None

    if is_absolute:
        url = source.replace("www.", "")
    elif source.startswith("www."):
        url = urljoin("http://", source.replace("www.", ""))
    else:
        url = urljoin(base_url, source)

    parsed = urlparse(url)
    return f"{parsed.scheme}://{parsed.netloc}{parsed.path}"


def get_download_path(base_url: str, absolute_url: str, download_dir: str):
    if not base_url.endswith("/"):
        base_url += "/"

    relative_path = absolute_url.replace(base_url, "")
    download_path = Path(download_dir) / relative_path

    if not os.path.exists(download_path.parent):
        os.makedirs(download_path.parent)

    return download_path

In [ ]:
download_dir = "downloaded"
base_url = "http://pythonscraping.com"
url = "http://www.pythonscraping.com"

html = urlopen(url)
bs = BeautifulSoup(html, "html.parser")
download_list = bs.find_all(src=True)

for download in download_list:
    file_url = get_absolute_url(base_url, download["src"])
    if file_url:
        print(file_url)
        download_path = get_download_path(base_url, file_url, download_dir)
        urlretrieve(file_url, download_path)

In [3]:
with open("test.csv", "w+") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(("number", "number plus 2", "number times 2"))
    for i in range(10):
        writer.writerow((i, i + 2, i * 2))

In [67]:
def execute_sql_from_file(sql_file: str, db_name: str = "chapter6") -> None:
    conn = sqlite3.connect(f"../sqlite/db/{db_name}.db")
    cursor = conn.cursor()

    try:
        with open(f"../sqlite/sql/chapter6/{sql_file}", "r") as sqlFile:
            command = sqlFile.read()

        cursor.executescript(command)
        conn.commit()
    finally:
        cursor.close()
        conn.close()


def describe_table(table_name: str, db_name: str = "chapter6") -> None:
    conn = sqlite3.connect(f"../sqlite/db/{db_name}.db")
    cursor = conn.cursor()

    try:
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = cursor.fetchall()

        print(f"{db_name}.{table_name}")
        print("=" * 20)
        for column_info in columns_info:
            print(column_info)
        print()
    finally:
        cursor.close()
        conn.close()


def print_table_data(table_name: str, db_name: str = "chapter6") -> None:
    conn = sqlite3.connect(f"../sqlite/db/{db_name}.db")
    cursor = conn.cursor()

    try:
        cursor.execute(f"SELECT * FROM {table_name}")
        table_data = cursor.fetchall()

        print(f"{db_name}.{table_name}")
        print("=" * 20)
        for row in table_data:
            print(row)
        print()
    finally:
        cursor.close()
        conn.close()

In [54]:
execute_sql_from_file("create_pages_table.sql")
describe_table("pages")

(0, 'id', 'INTEGER', 0, None, 1)
(1, 'title', 'TEXT', 0, None, 0)
(2, 'content', 'TEXT', 0, None, 0)
(3, 'created', 'TIMESTAMP', 0, 'CURRENT_TIMESTAMP', 0)


In [55]:
execute_sql_from_file("insert_3items.sql")
print_table_data("pages")

pages
(1, 'test title 1', 'test content 1', '2024-02-03 10:13:47')
(2, 'test title 2', 'test content 2', '2024-02-03 10:13:47')
(3, 'test title 3', 'test content 3', '2024-02-03 10:13:47')


In [56]:
execute_sql_from_file("update_item3.sql")
print_table_data("pages")

pages
(1, 'test title 1', 'test content 1', '2024-02-03 10:13:47')
(2, 'test title 2', 'test content 2', '2024-02-03 10:13:47')
(3, '新しいタイトル3', '新しいコンテンツ3', '2024-02-03 10:13:47')


In [57]:
execute_sql_from_file("delete_item3.sql")
print_table_data("pages")

pages
(1, 'test title 1', 'test content 1', '2024-02-03 10:13:47')
(2, 'test title 2', 'test content 2', '2024-02-03 10:13:47')


In [58]:
class Article:
    def __init__(self, title: str, content: str, links: List[str]) -> None:
        self.title = title
        self.content = content
        self.links = links


def scrape_wikipedia_with_db(start_url: str) -> None:
    print(start_url)

    connection = sqlite3.connect("../sqlite/db/chapter6.db")
    cursor = connection.cursor()

    try:
        article = describe_article(start_url)

        store(cursor, article)

        while len(article.links) > 0:
            new_link = random.choice(article.links)
            print(f"--> {new_link}")

            article = describe_article(new_link)
            store(cursor, article)
    except KeyboardInterrupt:
        print("KeyboardInterrupt: 処理を中断しました")
        cursor.connection.commit()
    finally:
        cursor.close()
        connection.close()


def describe_article(article_url: str) -> Article:
    with urlopen(f"http://en.wikipedia.org{article_url}") as html:
        bs = BeautifulSoup(html, "html.parser")

    title = bs.find("h1").get_text()  # type: ignore
    body = bs.find("div", id="mw-content-text")
    content = body.find_all("p")[1].get_text()  # type: ignore

    pattern = r"^(/wiki/)((?!:).)*$"
    link_tags = body.find_all("a", href=re.compile(pattern))  # type: ignore
    links = [link_tag.attrs.get("href") for link_tag in link_tags]  # type: ignore

    return Article(title, content, links)  # type: ignore


def store(cursor: sqlite3.Cursor, article: Article) -> None:
    cursor.execute(
        "INSERT INTO pages (title, content) VALUES (?, ?);",
        (article.title, article.content),
    )
    cursor.connection.commit()

In [59]:
scrape_wikipedia_with_db("/wiki/Kevin_Bacon")

/wiki/Kevin_Bacon
--> /wiki/Steven_Yeun
--> /wiki/James_Woods
--> /wiki/Richard_Crenna
--> /wiki/Hollywood_Walk_of_Fame
KeyboardInterrupt: 処理を中断しました


In [60]:
print_table_data("pages")

pages
(1, 'test title 1', 'test content 1', '2024-02-03 10:13:47')
(2, 'test title 2', 'test content 2', '2024-02-03 10:13:47')
(4, 'Kevin Bacon', '\n', '2024-02-03 10:13:58')
(5, 'Steven Yeun', 'Yeun Sang-yeop[1] (Korean: 연상엽; born  December 21, 1983), known professionally as Steven Yeun (/jʌn/), is an American actor. Yeun initially rose to prominence for playing Glenn Rhee in the television series The Walking Dead (2010–2016). He earned critical acclaim for starring in the thriller Burning (2018) and drama Minari (2020). The latter earned him a nomination for the Academy Award for Best Actor, becoming the first Asian American actor to do so.[2] Time magazine named him one of the 100 most influential people in the world in 2021.[3] In 2023, he starred in the dark comedy series Beef (2023), for which he won two Primetime Emmy Awards and a Golden Globe Award.\n', '2024-02-03 10:14:00')
(6, 'James Woods', "James Howard Woods (born April 18, 1947) is an American actor. He is known for fas

In [108]:
execute_sql_from_file("create_links_table.sql", db_name="wikipedia")
describe_table("pages", db_name="wikipedia")
describe_table("links", db_name="wikipedia")

wikipedia.pages
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'url', 'TEXT', 1, None, 0)
(2, 'created', 'TIMESTAMP', 1, 'CURRENT_TIMESTAMP', 0)

wikipedia.links
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'from_page_id', 'INTEGER', 1, None, 0)
(2, 'to_page_id', 'INTEGER', 1, None, 0)
(3, 'created', 'TIMESTAMP', 1, 'CURRENT_TIMESTAMP', 0)



In [109]:
def insert_page(cursor: sqlite3.Cursor, url: str) -> int:
    cursor.execute("SELECT * FROM pages WHERE url = ?", (url,))
    registered_record = cursor.fetchone()

    ## 登録済みの場合は登録済みレコードのidを返す
    if registered_record:
        return registered_record[0]

    ## 未登録の場合は登録後、登録したレコードのidを返す
    cursor.execute("INSERT INTO pages (url) VALUES (?)", (url,))
    cursor.connection.commit()
    return cursor.lastrowid  # type: ignore


def insert_link(
    cursor: sqlite3.Cursor,
    from_page_id: int,
    to_page_id: int,
) -> None:
    cursor.execute(
        "SELECT * FROM links WHERE from_page_id = ? AND to_page_id = ?",
        (from_page_id, to_page_id),
    )
    registered_record = cursor.fetchone()

    ## 登録済みの場合は何もしない
    if registered_record:
        return

    ## 未登録の場合は登録
    cursor.execute(
        "INSERT INTO links (from_page_id, to_page_id) VALUES (?, ?)",
        (from_page_id, to_page_id),
    )
    cursor.connection.commit()


def load_visited_page_urls(cursor: sqlite3.Cursor) -> List[str]:
    cursor.execute("SELECT * FROM pages")
    visited_page_urls = [row[1] for row in cursor.fetchall()]

    return visited_page_urls


def visit_recursively(
    page_url: str,
    cursor: sqlite3.Cursor,
    visited_page_urls: List[str],
    recursion_level: int = 1,
) -> None:
    # 訪問を記録
    print(f"深さ{recursion_level}: {page_url}")
    visited_page_urls += [page_url]

    # 探索範囲は2次リンクまで
    if recursion_level > 1:
        return

    # ページ内リンクの収集
    with urlopen(f"http://en.wikipedia.org{page_url}") as html:
        bs = BeautifulSoup(html, "html.parser")
    pattern = "^(/wiki/)((?!:).)*$"
    link_tags = bs.find_all("a", href=re.compile(pattern))
    links = [link_tag.attrs.get("href") for link_tag in link_tags]

    # ページURL/リンクの格納
    from_page_id = insert_page(cursor, page_url)
    for link in links:
        to_page_id = insert_page(cursor, link)
        insert_link(cursor, from_page_id, to_page_id)

        # 訪問済みのページであればスキップ
        if link in visited_page_urls:
            continue

        visit_recursively(
            page_url=link,
            cursor=cursor,
            visited_page_urls=visited_page_urls,
            recursion_level=recursion_level + 1,
        )

In [110]:
connection = sqlite3.connect("../sqlite/db/wikipedia.db")
cursor = connection.cursor()

try:
    visited_page_urls = load_visited_page_urls(cursor)
    visit_recursively(
        page_url="/wiki/Kevin_Bacon",
        cursor=cursor,
        visited_page_urls=visited_page_urls,
    )
except KeyboardInterrupt:
    print("処理を中断しました")
finally:
    cursor.close()
    connection.close()

深さ1: /wiki/Kevin_Bacon
深さ2: /wiki/Main_Page
深さ2: /wiki/Kevin_Bacon_(disambiguation)
深さ2: /wiki/Philadelphia
深さ2: /wiki/Kevin_Bacon_filmography
深さ2: /wiki/Kyra_Sedgwick
深さ2: /wiki/Sosie_Bacon
深さ2: /wiki/Edmund_Bacon_(architect)
深さ2: /wiki/Michael_Bacon_(musician)
深さ2: /wiki/Leading_man
深さ2: /wiki/Character_actor
深さ2: /wiki/Golden_Globe_Award
深さ2: /wiki/Screen_Actors_Guild_Award
深さ2: /wiki/Primetime_Emmy_Award
深さ2: /wiki/National_Lampoon%27s_Animal_House
深さ2: /wiki/Footloose_(1984_film)
深さ2: /wiki/Diner_(1982_film)
深さ2: /wiki/JFK_(film)
深さ2: /wiki/A_Few_Good_Men
深さ2: /wiki/Apollo_13_(film)
深さ2: /wiki/Mystic_River_(film)
深さ2: /wiki/Frost/Nixon_(film)
深さ2: /wiki/Friday_the_13th_(1980_film)
深さ2: /wiki/Tremors_(1990_film)
深さ2: /wiki/The_River_Wild
深さ2: /wiki/The_Woodsman_(2004_film)
深さ2: /wiki/Crazy,_Stupid,_Love
深さ2: /wiki/Patriots_Day_(film)
深さ2: /wiki/Losing_Chase
深さ2: /wiki/Loverboy_(2005_film)
深さ2: /wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Miniseries_or_Television_Film
深さ2: /wik

In [111]:
print_table_data("pages", db_name="wikipedia")
print_table_data("links", db_name="wikipedia")

wikipedia.pages
(1, '/wiki/Kevin_Bacon', '2024-02-03 15:41:55')
(2, '/wiki/Main_Page', '2024-02-03 15:41:55')
(3, '/wiki/Kevin_Bacon_(disambiguation)', '2024-02-03 15:41:55')
(4, '/wiki/Philadelphia', '2024-02-03 15:41:55')
(5, '/wiki/Kevin_Bacon_filmography', '2024-02-03 15:41:55')
(6, '/wiki/Kyra_Sedgwick', '2024-02-03 15:41:55')
(7, '/wiki/Sosie_Bacon', '2024-02-03 15:41:55')
(8, '/wiki/Edmund_Bacon_(architect)', '2024-02-03 15:41:55')
(9, '/wiki/Michael_Bacon_(musician)', '2024-02-03 15:41:55')
(10, '/wiki/Leading_man', '2024-02-03 15:41:55')
(11, '/wiki/Character_actor', '2024-02-03 15:41:55')
(12, '/wiki/Golden_Globe_Award', '2024-02-03 15:41:55')
(13, '/wiki/Screen_Actors_Guild_Award', '2024-02-03 15:41:55')
(14, '/wiki/Primetime_Emmy_Award', '2024-02-03 15:41:55')
(15, '/wiki/National_Lampoon%27s_Animal_House', '2024-02-03 15:41:55')
(16, '/wiki/Footloose_(1984_film)', '2024-02-03 15:41:55')
(17, '/wiki/Diner_(1982_film)', '2024-02-03 15:41:55')
(18, '/wiki/JFK_(film)', '2024-0